# Quantify Day-to-Day Multidirectional Spread 

## Setup Connection to Drive

In [ ]:
from google.colab import drive # import drive from google colab
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
rootPath = "/content/drive/My Drive/California FireTrends (2012-2020)"
os.chdir(rootPath)

<b>Multi-Directional Spread</b>
<n>Use 'trace-back' and 'back-fill' method where shortest distance from points across firefront back to previous day shared line is calculated.

1. Get smoothed interpolated daily perimeters 
2. Get shared fire-line Day1 & Day 2
3. Points along front line and shared line 
4. Find shortest distance from front line back to shared line 

For days > 1 then shared fireline can be past two days. 

In [ ]:
def lstFiles(rootPath, ext):
  '''
  retrieve file path + names based on extension
  '''
  file_list = []
  root = rootPath
  for path, subdirs, files in os.walk(root):
      for names in files: 
          if names.endswith(ext):
              file_list.append(os.path.join(path,names))
  return file_list

def createFolder(rootPath, folderName): 
  '''
  Create new folder in root path 
  '''
  folderPath = os.path.join(rootPath, folderName) 
  if not os.path.exists(folderPath):
      os.makedirs(folderPath)
  return folderPath 


def listFiles(rootPath):
  '''
  retrieve file path + names 
  '''
  file_list = []
  root = rootPath
  for path, subdirs, files in os.walk(root):
    for names in files: 
      file_list.append(os.path.join(path,names))
  return file_list

In [ ]:
import pandas as pd
from scipy.spatial.distance import cdist

def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]

In [ ]:
import math
from math import radians, degrees, sin, cos, asin, acos, sqrt
# def calculateDistance(pointA, pointB):
#     lat1 = math.radians(pointA[0])
#     lat2 = math.radians(pointB[0])
#     lon1 = math.radians(pointA[1])
#     lon2 = math.radians(pointB[1])
#     return 6371 * (
#         acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon1 - lon2))
#     )

def calculate_initial_compass_bearing(pointA, pointB):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `pointA: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `pointB: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(pointA) != tuple) or (type(pointB) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(pointA[0])
    lat2 = math.radians(pointB[0])

    diffLong = math.radians(pointB[1] - pointA[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing

In [ ]:
pointA = (41.01293623, -122.04922577)
pointB = (41.01176303, -122.03467066)
calculate_initial_compass_bearing(pointB, pointA)

276.1020670509466

In [ ]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
import folium
import rtree

## Get Derived Fire Info

In [ ]:
fireInfo = pd.read_csv('Products/DerivedFireInfo_Total.csv', index_col=0)
fireInfo

## Filter to 85% of Cumulative Burn Area

In [ ]:

def filterOutPerimeter(path):
  # read in derived geo with geopandas 
  new_path = path.replace('By_Fire', 'By_Fire_Filtered')
  if os.path.exists(new_path) == False: 
    # print(path)
    gdf = gpd.read_file(path)

    # get aggregated fire count 
    fireCount = gdf.groupby(['Fire', 'Year'])
    cumudf = fireCount.agg({'JulianDay' : ['count'] , 'Area (ha)' : ['sum']}).reset_index()
    cumudf.columns = ['Fire', 'Year', 'Original Total Days', 'Original Total Area']
    
    # calculate percent cumulative burn 
    newDF = gdf.merge(cumudf, on=['Fire', 'Year'])
    newDF['Ratio'] = newDF['Area (ha)']  / newDF['Original Total Area'] 
    newDF['cumsum'] = newDF.groupby(['Fire', 'Year'])['Ratio'].cumsum() 

    # filter, keep top 85% 
    newDF = newDF[(newDF['Original Total Days'] < 6) | (newDF['cumsum'] <= .85)]
    newCount = newDF.groupby(['Fire', 'Year'])
    newCount = newCount.agg({'JulianDay' : ['count'] , 'Area (ha)' : ['sum']}).reset_index()
    newCount.columns = ['Fire', 'Year', 'Filtered Total Days', 'Filtered Total Area']
    new = newDF.merge(newCount, on=['Fire', 'Year'])
     
    # save filtered geometries
    file_nm = os.path.basename(new_path)
    new_directory = new_path.replace(file_nm, '')

    if not os.path.exists(new_directory):
        os.makedirs(new_directory)

    new.to_file(new_path)

    print('filtered', file_nm)

In [ ]:
interFiles = lstFiles(os.path.join(r'Products', 'By_Fire'), '.shp')

In [ ]:
for shp_path in interFiles: 
  filtered_df = filterOutPerimeter(shp_path)

In [ ]:
filteredFiles = lstFiles(os.path.join(r'Products', 'By_Fire_Filtered'), '.shp')

In [ ]:
filteredFiles[0]

'Products/By_Fire_Filtered/2018/Ferguson/Ferguson_2018_NAT.shp'

## Get Daily Spread Info

Distance, Direction


### Get Ignition

In [ ]:
# rename 
ignition_files = listFiles('Ignition/Ignition_Points')

for fls in ignition_files:
  pth = os.path.dirname(fls)
  oldname = os.path.basename(fls)
  newname = oldname.strip().replace(' ', '')
  os.rename(os.path.join(pth, oldname), os.path.join(pth, newname))
  print(newname)

In [ ]:
def getIgnitionDF(poly_path):
  ignition_pth = poly_path.replace('Products', 'Ignition')
  ignition_pth = ignition_pth.replace('By_Fire', 'Ignition_Points')
  ignition_pth = ignition_pth.replace('NAT', 'IgnitionPnts')
  ignition_gdf = gpd.read_file(ignition_pth)
  return ignition_gdf

### Generate Points Along Bounds

In [ ]:
def generatePointsAlongLine(firebounds_df):
  points_along_line = []
  for len in range(1, 1000, 1):
    len_per = len/1000
    points_along_line.append(firebounds_df.boundary.interpolate(len_per, normalized=True)[0])
  
  d = {'geometry': points_along_line}
  boundsToPoints_df = gpd.GeoDataFrame(d, crs= firebounds_df.crs)
  return boundsToPoints_df

### Visualize Spread Vectors


In [ ]:
def visualizeVectors(df, poly_df):
  start_gdf = gpd.GeoDataFrame(df, geometry='start_coord').set_crs('EPSG:3310').reset_index(drop=True)
  end_gdf = gpd.GeoDataFrame(df, geometry='end_coord').set_crs('EPSG:3310').reset_index(drop=True)
  line_gdf = gpd.GeoDataFrame(df, geometry='line_coord').set_crs('EPSG:3310').reset_index(drop=True)
  
  line_gdf_clipped = gpd.clip(line_gdf, poly_df.geometry.buffer(0.01))
  line_gdf_clipped['line_distance'] = line_gdf_clipped['line_coord'].length

  # base = start_gdf.plot(color='black', markersize=1)
  # end_gdf.plot(ax=base, marker='o', color='green', markersize=1) 
  # line_gdf_clipped.plot(ax=base, color='red', alpha=0.4)  
  return line_gdf_clipped

### First Day Spread

In [ ]:
import geopandas as gpd
from shapely.geometry import mapping
from shapely.ops import nearest_points

def firstDaySpread(day_num, poly_path, firstDay):
  firefront_df = generatePointsAlongLine(firstDay)
  startPnt_df = getIgnitionDF(poly_path)

  nearest_df = findNearestPoint(startPnt_df.geometry, firefront_df.geometry)

  df = getSpreadInfo(nearest_df, firstDay.Date[0], firstDay.Year[0], firstDay.Fire[0]) 

  return df

In [ ]:
def generateDayPairs(gdf):
  days = gdf['JulianDay'].tolist()
  dayPairs = list(zip(days, days[1:] + days[:1])) 
  dayPairs = dayPairs[:-1] 
  return dayPairs


In [ ]:
from shapely.ops import nearest_points

def findNearestPoint(prevDay_pnts, currDay_pnts):
  # unary union of the gpd2 geomtries 
  prevDay_Multipoint = prevDay_pnts.geometry.unary_union
  
  def near(pnt1, pts=prevDay_Multipoint):
      # find the nearest point and return the corresponding Place value
      curr_pnt, closest_pnt = [o.wkt for o in nearest_points(pnt1, pts)]
      return curr_pnt, closest_pnt
      
  nearest = currDay_pnts.apply(lambda x: near(x))

  current_Point = [] 
  closest_Point = [] 

  for A, B in nearest: 
    current_Point.append(A)
    closest_Point.append(B)

  nearstPointsPair = pd.DataFrame({'Current_Day' : current_Point ,'Nearest_Point' : closest_Point })

  return nearstPointsPair

### Create GeoDataFrame from listo of Points (XY)

In [ ]:
from shapely import wkt

def createGeoDataFrame(pointList):
  dic = {'geometry' : map(wkt.loads, pointList)}
  df =  gpd.GeoDataFrame(dic, geometry='geometry')
  return df

### Calculate distance and direction, save into df

In [ ]:
import shapely.wkt

def calculateDistDir(date, year, fire, currentPoints, startPoints):

  distance = [] 
  direction = [] 
  start_coord = [] 
  end_coord = [] 
  dt = [] 
  fire_name = []
  yr = []
  line_coord = []

  for endpnt, startpnt in list(zip(currentPoints, startPoints)):
      # get distance
      distance.append(startpnt.distance(endpnt))
      
      # get direction 
      start_pnt_coord = (startpnt.x, startpnt.y)
      end_pnt_coord = (endpnt.x, endpnt.y)
      direction.append(calculate_initial_compass_bearing(start_pnt_coord, end_pnt_coord))

      start_coord.append(startpnt)
      end_coord.append(endpnt)
      line_coord.append(LineString([(startpnt.x, startpnt.y), (endpnt.x, endpnt.y)]))
      
      
      dt.append(date)
      fire_name.append(fire)
      yr.append(year)
      
  daily_spread_Info = pd.DataFrame({'Fire' : fire_name, 'Year':yr, 'Date': dt,
                                      'distance': distance, 'direction':direction, 
                                      'start_coord': start_coord, 'end_coord':end_coord,
                                      'line_coord': line_coord})
  return daily_spread_Info

### Extract spread info

In [ ]:
from shapely.geometry import Point, LineString
from shapely import geometry, ops

def getSpreadInfo(nearstPointsPair, date, year, fire): 
  PNT_A = nearstPointsPair['Current_Day'].tolist()
  PNT_B = nearstPointsPair['Nearest_Point'].tolist()
  
  currentPoints = map(wkt.loads, PNT_A)
  startPoints = map(wkt.loads, PNT_B)

  backfill_df = calculateDistDir(date, year, fire, currentPoints, startPoints)
  

  # for points that overlap; backfill from previous day to fire front 
  start_gdf = gpd.GeoDataFrame(backfill_df, geometry='start_coord').set_crs('EPSG:3310')
  end_gdf = gpd.GeoDataFrame(backfill_df, geometry='end_coord').set_crs('EPSG:3310')

  new_end_points = end_gdf[end_gdf['distance'] > 100]['end_coord']
  new_start_points = start_gdf[start_gdf['distance'] < 100]['start_coord']

  if len(new_start_points) > 1: 
    new_nearest_df = findNearestPoint(new_end_points, new_start_points)
    
    new_PNT_A = new_nearest_df['Current_Day'].tolist()
    new_PNT_B = new_nearest_df['Nearest_Point'].tolist()

    new_currentPoints = map(wkt.loads, new_PNT_A)
    new_startPoints = map(wkt.loads, new_PNT_B)
    
    frontfill_df = calculateDistDir(date, year, fire, new_startPoints, new_currentPoints)

    frames = [frontfill_df, backfill_df]
    result = pd.concat(frames)

    return result
  else: 
    return backfill_df

### Final magnitude

In [ ]:
def getFinalMagnitude(fire_poly):
  gdf = gpd.read_file(fire_poly)
  # gdf.to_crs('EPSG:4326')
  gdf = gdf.sort_values(by=['JulianDay'])

  daypairs = generateDayPairs(gdf) 
  for day_num in range(-1, len(daypairs)-1):
    if day_num == -1 : 
      # get first day df 
      firstDay = gdf[:1]
      df = firstDaySpread(day_num, fire_poly, firstDay)
      df = visualizeVectors(df, firstDay) 
      # append df
      complete_df_byFire.append(df)
    
    else: 
      prevDay = gdf[day_num:day_num+1].reset_index()
      currDay = gdf[day_num+1:day_num+2].reset_index()
            
      prevDay_pnts = generatePointsAlongLine(prevDay)
      currDay_pnts = generatePointsAlongLine(currDay)

      nearest_df = findNearestPoint(prevDay_pnts, currDay_pnts.geometry)

      df = getSpreadInfo(nearest_df, currDay.Date[0], currDay.Year[0], currDay.Fire[0])
      
      # append df 
      
      df = visualizeVectors(df, currDay.reset_index())
      complete_df_byFire.append(df)
  
  return complete_df_byFire

## Run Complete Spread 

In [ ]:
# [TODO] fix direction (crs decimal degrees)

interFiles = lstFiles(os.path.join(r'Products', 'By_Fire'), '.shp')
outpath = 'MultiSpread/CSV'

for fire_poly in interFiles:
  nm = os.path.basename(fire_poly).replace('shp', 'csv')
  print(nm)
  if os.path.exists(os.path.join(outpath, nm)) == False:
    try: 
      complete_df_byFire = []
      final_df = pd.concat(getFinalMagnitude(fire_poly)).reset_index(drop=True)
      final_df.to_csv(os.path.join(outpath, nm))
    except: 
      print('error')

In [ ]:
# final_df = gpd.GeoDataFrame(final_df, geometry='start_coord')

#   new_outpath = fire_poly.replace('Products', 'MultiSpread')
#   new_outpath = new_outpath.replace('By_Fire', 'Vectors')
#   new_outpath = new_outpath.replace('NAT', 'MultiSpreadVectors')
#   new_outpath = new_outpath.replace('shp', 'geojson')

#   folderPath = os.path.dirname(new_outpath)
#   if not os.path.exists(folderPath):
#       os.makedirs(folderPath)

#   print(new_outpath)
#   final_df.to_file(new_outpath, driver='GeoJSON')

#   new_csv = new_outpath.replace('Vectors', 'CSV')
#   new_csv = new_csv.replace('shp', 'csv')

#   csvfolderPath = os.path.dirname(new_csv)
#   if not os.path.exists(csvfolderPath):
#       os.makedirs(csvfolderPath)


## Merge Into One CSV
Merge, upperquantile, median

In [ ]:
outpath = 'MultiSpread/CSV'
csv_list = lstFiles(outpath, '.csv')

In [ ]:
len(csv_list)

471

In [ ]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in csv_list])
#export to csv
combined_csv.to_csv( "MultiSpread/combined_magnitude_2012_2020.csv", index=False)

In [ ]:
from shapely import wkt
combined_csv['line_coord'] = combined_csv['line_coord'].apply(wkt.loads)
combined_csv['start_coord'] = combined_csv['start_coord'].apply(wkt.loads)
combined_csv['end_coord'] = combined_csv['end_coord'].apply(wkt.loads)

## Fix Direction

In [ ]:
START_gpd = gpd.GeoDataFrame(combined_csv, geometry='start_coord', crs='EPSG:3310').reset_index(drop=True)
END_gpd = gpd.GeoDataFrame(combined_csv, geometry='end_coord', crs='EPSG:3310').reset_index(drop=True)

In [ ]:
START_gpd = START_gpd.to_crs('EPSG:4326')
END_gpd = END_gpd.to_crs('EPSG:4326')

In [ ]:
start_coord_list = list(zip(START_gpd.geometry.x, START_gpd.geometry.y))
end_coord_list = list(zip(END_gpd.geometry.x, END_gpd.geometry.y))

In [ ]:
directions = [calculate_initial_compass_bearing(x,y) for x,y in zip(start_coord_list,end_coord_list)]

In [ ]:
combined_csv['direction'] = directions
combined_csv['distance'] = combined_csv['line_distance']

In [ ]:
combined_csv = combined_csv.drop(columns=['line_distance'])

In [ ]:
combined_csv.to_csv( "MultiSpread/combined_magnitude_2012_2020.csv", index=False)

In [ ]:
! pip install windrose

In [ ]:
from windrose import WindroseAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd 
import os
import seaborn as sns
from pylab import savefig

In [ ]:
def windRosePlot(df, figurePath): 
    ws = df['Direction'].values
    wd = df['line_distance'].values
    ax = WindroseAxes.from_ax()
    ax.contourf(ws, wd, cmap=cm.hot)
    ax.set_legend()
    ax.figure.savefig(figurePath + fr + "_" + str(yr) + '.png', dpi = 400)
    ax.figure.clf()

### Weighted Average Based on Distance

In [ ]:
# Weighted Average DEF 
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

### Upper and Lower Quartiles

In [ ]:
# find quartile 
def quartiles(dataPoints):
    # check the input is not empty
    # 1. order the data set
    sortedPoints = sorted(dataPoints)
    # 2. divide the data set in two halves
    mid = len(sortedPoints) // 2 # uses the floor division to have integer returned
    if (len(sortedPoints) % 2 == 0):
        # even
        lowerQ = np.median(sortedPoints[:mid])
        upperQ = np.median(sortedPoints[mid:])
    else:
        # odd
        lowerQ = np.median(sortedPoints[:mid])  # same as even
        upperQ = np.median(sortedPoints[mid+1:])
    return (lowerQ, upperQ)

In [ ]:
FinalDF = pd.read_csv(r'MultiSpread/combined_magnitude_2012_2020.csv', index_col=0)

In [ ]:
# Upper/Lower Quartiles
lowerQuart = FinalDF.groupby(['Fire', "Date"], group_keys=False)['distance'].apply(lambda x: quartiles(x)[0]).reset_index()
upperQuart = FinalDF.groupby(['Fire', "Date"], group_keys=False)['distance'].apply(lambda x: quartiles(x)[1]).reset_index()

In [ ]:
quart = lowerQuart.merge(upperQuart, on=['Fire', "Date"])
quart.columns = ['Fire', "Date", 'Magnitude (lowerQ)', 'Magnitude (upperQ)']

In [ ]:
# Weighted Average 
weighted_Avg = FinalDF.groupby(['Fire', "Date"]).apply(wavg, "direction", "distance").reset_index()
weighted_Avg.columns = ['Fire', 'Date', '(DIR_Weighted)']

In [ ]:
# Get Max, Median, STD of Distance
f = {'distance': ['max', 'median', 'std']}   
distance_stats = FinalDF.groupby(['Fire', "Date"]).agg(f).reset_index()
distance_stats.columns = ['Fire', 'Date', 'Magnitude (max)','Magnitude (median)','Magnitude (stdv)']

In [ ]:
# Direction at Max Distance
dir_maxdist = FinalDF.loc[df.reset_index().groupby(['Fire', "Date"])['distance'].idxmax()].reset_index()
dir_maxdist = dir_maxdist[['Fire', 'Date', 'direction']]
dir_maxdist.columns = ['Fire', 'Date', 'DIR_MaxDist']

In [ ]:
finalDF = weighted_Avg.merge(distance_stats, on=['Fire', "Date"])
finalDF = finalDF.merge(dir_maxdist, on=['Fire', "Date"]).round(3)
finalDF = finalDF.merge(quart, on=['Fire', "Date"]).round(3)

In [ ]:
finalDF['Magnitude (median)'] = finalDF['Magnitude (median)'].astype(str)
finalDF['Magnitude (stdv)'] = finalDF['Magnitude (stdv)'].astype(str)

In [ ]:
finalDF['DIST_MEDSTD'] = finalDF[['Magnitude (median)', 'Magnitude (stdv)']].apply(lambda x: ' ± '.join(x[x.notnull()]), axis = 1)

## Save Final Magnitude File

In [ ]:
finalDF.to_csv('MultiSpread/daily_magnitude_2012_2020.csv', index=False)

In [ ]:
finalDF

### Create Plots, csv files for each fireyear instance

In [ ]:
FinalDF = pd.read_csv(r'MultiSpread/combined_magnitude_2012_2020.csv', index_col=0)

FR_YR_List = list(set(list(zip(FinalDF['Fire'], FinalDF['Year']))))

In [ ]:
len(FR_YR_List)

471

In [ ]:
sns.set(style="whitegrid")
palette = sns.color_palette("mako_r", 3)
rootPath = r"MultiSpread/By_Fire" 
for fr, yr in FR_YR_List[:1]: 
    print(fr, yr)
    
    flname = fr + "_" + str(yr) + "_FinalAll.csv"
    CSVPath = os.path.join(rootPath, str(yr), fr, "FinalCSV", flname)
    figurePath = createFolder(os.path.join(rootPath, str(yr), fr), "Figures")

    df = FinalDF[(FinalDF['Fire'] == fr) & (FinalDF['Year'] == yr)]


Blake 2015


In [ ]:
sns.set(style="whitegrid")
palette = sns.color_palette("mako_r", 3)
rootPath = r"MultiSpread" 
for fr, yr in FR_YR_List: 
    print(fr, yr)
    flname = fr + "_" + str(yr) + "_FinalAll.csv"
    CSVPath = os.path.join(rootPath, str(yr), fr, "FinalCSV", flname)
    figurePath = createFolder(os.path.join(rootPath, str(yr), fr), "Figures")
    
    df = pd.read_csv(CSVPath, index_col = 0) 
    windRosePlot(df, figurePath)
    csvPath = createFolder(rootPath, "SummaryCSV")
    yrPath = createFolder(csvPath, str(yr))
    finalDF.to_csv(os.path.join(yrPath, csvName))
    # Plot the responses for different events and regions 
    matplotlib.pyplot.close('all')
    finalDF['log(Area)'] = np.log(finalDF['Area (ha)'])
    melted = pd.melt(finalDF, id_vars=['JulianDay'], value_vars=['log(Area)', 'Magnitude (max)', 'Magnitude (median)'],
        var_name='Value Type', value_name='Values')
    melted['Values'] = melted['Values'].astype('float64')
    snsPLOT = sns.lineplot(x="JulianDay", y="Values", hue = 'Value Type', style = 'Value Type', palette = palette, data=melted)
    figure = snsPLOT.get_figure()    
    figure.savefig(figurePath + fr + "_" + str(yr) + "_timeseries.png", dpi = 400)
    matplotlib.pyplot.close('all')
    matplotlib.pyplot.clf()